In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##Model try

In [ ]:
from google.colab import drive
import numpy as np
import os
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# Define classes and their corresponding labels
class_names = {0: 'Dirty Mark', 1: 'Cloth', 2: 'Ruler'}
num_classes = len(class_names)

# Load annotations
annotations = {
    "image_paths": [],
    "labels": []
}

# Change the directory to where your annotation files are located
annotation_files = os.listdir("/content/gdrive/MyDrive/QC-BDMLI-Assessment/data/labels")

for file in annotation_files:
    with open(os.path.join("/content/gdrive/MyDrive/QC-BDMLI-Assessment/data/labels", file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.split()
            annotations["image_paths"].append(os.path.join("/content/gdrive/MyDrive/QC-BDMLI-Assessment/data/images", file[:-3] + "jpg"))
            annotations["labels"].append(int(parts[0]))

# Load images and annotations
images = []
labels = []
for i, image_path in enumerate(annotations["image_paths"]):
    image = cv2.imread(image_path)
    # Preserve aspect ratio when resizing
    aspect_ratio = image.shape[1] / image.shape[0]
    new_height = 224
    new_width = int(new_height * aspect_ratio)
    image = cv2.resize(image, (new_width, new_height))
    # Pad the image to make it square
    pad_left = (max(new_height, new_width) - new_width) // 2
    pad_right = max(new_height, new_width) - new_width - pad_left
    pad_top = (max(new_height, new_width) - new_height) // 2
    pad_bottom = max(new_height, new_width) - new_height - pad_top
    image = cv2.copyMakeBorder(image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    images.append(image)
    labels.append(annotations["labels"][i])

# Convert lists to numpy arrays
images = np.array(images)
labels = to_categorical(labels, num_classes=num_classes)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        layers.RandomBrightness(0.2),
    ]
)

# Define and compile the model
base_model = EfficientNetB0(include_top=False, input_shape=(max(new_height, new_width), max(new_height, new_width), 3))
base_model.trainable = True

model = keras.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tune the model
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Save the model
model.save("/content/gdrive/MyDrive/object_detection_model_v2.h5")
